In [1]:
!pip install openai tiktoken pypdf2 pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [245]:
import numpy as np
import pandas as pd
import os
import json
import requests

from datetime import datetime

import PyPDF2
from tqdm import tqdm
import time

import openai
from openai import OpenAI
import tiktoken

from pinecone import Pinecone, ServerlessSpec
#import pinecone

openai.api_key = 'your-openai-key'
pinecone_api_key = 'your-pinecone-key'

# Instantiate the OpenAI and Pinecone clients
client = OpenAI(api_key=openai.api_key)
pc = Pinecone(api_key=pinecone_api_key)

In [247]:
#!pip list

In [4]:
# Load the tokenizer corresponding to the GPT 3.5/4 models
tokenizer = tiktoken.get_encoding("cl100k_base")

# Sample string
text = "This is a sample text to count the number of tokens."

# Encode the string to get tokens
tokens = tokenizer.encode(text)

# Count the number of tokens
num_tokens = len(tokens)

print(f"The number of tokens in the string is: {num_tokens}")

The number of tokens in the string is: 12


#Data Ingestion

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
os.chdir('/content/drive/MyDrive/Prophet')
os.getcwd()

In [6]:
os.chdir('/content/drive/MyDrive/Interview Practice/Prophet')
os.getcwd()

'/content/drive/MyDrive/Interview Practice/Prophet'

In [7]:
doc_path = os.getcwd() + '/equifax_2018_Q1_10K.pdf'
pages, token_counts = {}, {}
#full_doc_text = ''

with open(doc_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f'The document has {num_pages} pages.')
            for page_num in range(27): # we are just analysize part 1 of the report
                page = reader.pages[page_num]
                text = page.extract_text()
                #full_doc_text += text
                pages[page_num + 1] = text
                num_tokens = len(tokenizer.encode(text))
                token_counts[page_num + 1] = num_tokens
                #print(f'The document has {num_tokens} tokens.')

The document has 135 pages.


In [ ]:
pages[5]

' \nOUR BUSINESS STRATEGY\n \nData is at the core of our value proposition and the protection and safeguarding of that information is paramount. Our strategic objective is to be the global leader in\ninformation solutions that creates unparalleled insights to solve customer challenges. Leveraging our extensive resources, we deliver differentiated decisions through a broad\nand diverse set of data assets, sophisticated analytics and proprietary decisioning technology. Our long-term corporate strategy is driven by the following imperatives:\n•\nServe as a trusted steward and advocate for consumers and our customers.\n This includes protecting and safeguarding the information we have using advanced\ndata security tools, techniques and processes in order to protect consumer specific information from fraudulent access. We also strive to continuously improve the\nconsumer and customer experience in our consumer and commercial offerings, anticipating and executing on regulatory initiatives, w

In [8]:
def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = text, model=model, dimensions=256).data[0].embedding

In [9]:
# Create Pinecode Index
'''
pc.create_index(
    name="prophet-equifax",
    dimension=256,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-west-2"
    )
)

for page_num, text in tqdm(pages.items()):
  embedding = get_embedding(text)
  pc.Index("prophet-equifax").upsert(vectors=[{"id": f"{page_num}", "values": embedding}])
'''

'\npc.create_index(\n    name="prophet-equifax",\n    dimension=256,\n    metric="cosine",\n    spec=ServerlessSpec(\n        cloud="aws",\n        region="us-west-2"\n    )\n)\n\nfor page_num, text in tqdm(pages.items()):\n  embedding = get_embedding(text)\n  pc.Index("prophet-equifax").upsert(vectors=[{"id": f"{page_num}", "values": embedding}])\n'

# Generate Question-Answer Dataset

In [10]:
# Come up with questions (80% specific to a single page, 15% multi-page retrieval, 5% not answerable from the document), percentages are rules of thumb

In [54]:
queries = ["What exactly was the security breach that occurred?", # Specfic, multi-doc retrieval
           "What was the extent of damage done as a result of the incident?", # Synthesized, multi-doc retrieval
           "What steps did Equifax take to rectify the hack and regain customer confidence?", # Specfic, multi-doc retrieval
           "How did the attacker succeed and what key information is left out in the document?", # Synthesized, multi-doc retrieval
           "Does Equifax operate in Tamil Nadu?", # no information
           "How does Equifax deal with fluctuating currency exchange values to maximize profits?", # pg 20
           "What are the primary countries in Europe that Equifax operates in?", # pg 7
           "Which revenue stream is usually the highest in the fourth quarter?", # pg 9
           "Which government agencies in Australia regulate Equifax's business activity?", # pg 13
           "What products and services were offered to customers in response to the breach?", # Specfic, multi-doc retrieval
           "How is Equifax's ability to grow its geographical footprint in emerging markets using joint ventures and acquisitions affected by the breach?", # pg 21
           "How many government contracts were suspended due to the breach?", # pg 22
           "How would Brexit impact Equifax's bottom-line?", # pg 23
           "How many class action suits were filed against Equifax following the breach?", # pg 25
           "TransUnion sued Equifax on what grounds?", # pg 25
           "Who were the criminals caused the Equifax security breach?",
           "What U.S. federal laws and regulations were violated due to the breach?",
           "Why does Equifax consider outsourcing their operations to be potentially harmful to their business?",
           "Why is the market for Equifax's products highly competitive?",
           "How has the percentage of revenue from Equifax's international clients changed over time?",
           "What products and services does USIS offer?",
           "Was a certain Equifax products or services adversely affected by the breach?",
           "What benefits do Equifax's products and services attempt to provide customers?",
           "What are the operating segments of Equifax's business?",
           "In which parts of the world are the Equifax's largest markets located?",
           "Does Equifax outsource its sales and marketing operations?",
           "How many companies does Equifax recieve payroll data from?",
           "Which Equifax service allows verifiers to verify an employee's employment status and income information?",
           "What products or services are listed as sources of competition for Equifax?",
           "How does the GDPR compare to the privacy laws of other countries?",
           "Were more consumers in U.K. or Canada potentially impacted by the cybersecurity breach?",
           "Can Indian customers pursue legal action against Equifax due to a data breach?",
           "How many employees did Equifax have by the end of 2017?",
           "Will all losses for the 2017 cybersecurity breach be covered by insurance?",
           "How can competitors decrease Equifax's sales and market share?",
           "Why does management believe the revenue from direct to consumer business will decline?",
           "What was the potential impact of the law passed in Ecuador at the end of 2017?",
           "How did Equifax expect the cyber attack to affect laws and regulations around their business?",
           "In what departments is Equifax having retention risks with key hires that affect operations?",
           "What is the address of Equifax's executive offices?"]

In [39]:
pages[25]

'Facility or acceleration of any obligations outstanding thereunder. Any event of default under the Senior Credit Facilities or Receivables Facility could result in a cross default\nunder our other outstanding debt obligations.\nChanges in interest rates could adversely affect our cost of capital and net income\n.\nRising interest rates, credit market dislocations and decisions and actions by credit rating agencies can affect the availability and cost of our funding and adversely\naffect our net income.\nOur business will suffer if we are not able to retain and hire key personnel.\nOur future success depends partly on the continued service of our key development, sales, marketing, executive and administrative personnel. Additionally, increased\nretention risk exists in certain key areas of our operations, such as IT and security, which require specialized skills, such as maintenance of certain legacy computer systems,\ndata security experts and analytical modelers. If we fail to retain

In [196]:
page_sources = [[3, 4, 16],
               [4, 17, 19],
               [4, 9, 16],
               [3, 16],
               [None],
               [21],
               [8],
               [10],
               [14],
               [9, 10, 4],
               [22],
               [23],
               [24],
               [26],
               [26],
               [3, 4],
               [12, 11, 3],
               [23],
               [10],
               [6],
               [7],
               [9, 10],
               [6, 5],
               [3],
               [6],
               [6],
               [9],
               [9],
               [10],
               [12],
               [3, 4],
               [14],
               [15],
               [16],
               [18],
               [19],
               [20],
               [20],
               [25],
               [25]]

expert_answers = ["Criminals used a web application vulnerability to gain access to sensitive Equifax customer data. This resulted in the disclosure of personally identifiable information belonging to 145.5 million U.S. customers.",
                 "The breach had a negative reputational impact on Equifax. This impact includes: poor revenue growth from 2016 to 2017, cancelled customer contracts, and suspended compliance certifications.",
                 "Equifax first acted immediately to stop the intrusion and hire an independent cybersecurity firm to investigate and determine the scope of the intrusion. They then reported the breach to affected customers and offered them free credit reports, monitoring, and identity theft protection. Equifax's board also made changes to the executive leadership team to involve leaders who prioritized data security and committed long-term resources to protecting customer data.",
                 "The attacker succeeded by exploiting a web application vulnerability. How that vulnerability was exploited, what the vulnerability was, and why it existed are key pieces of information that are not present in the given document.",
                 "This information is not available in the given document.",
                 "Equifax generally doesn't mitigate risks associated with fluctuating currency exchange values, though they may sometimes hedge against these risks.",
                 "The European countries that Equifax primarily operates in are Portugal, Spain, U.K.",
                 "Revenue generated from financial wealth asset products and data management services is usually the highest in fourth quarter.",
                 "The Australian government agencies that regulate Equifax's business activities are the Office of the Australian Information Commissioner (OAIC), Australian Competition and Consumer Commission (ACCC), Australian Retail Credit Association (ARCA), and Australian Securities and Investments Commission (ASIC).",
                 "In response to the breach, Equifax offered all U.S., Canada, and U.K. customers free identity theft protection, free credit file monitoring, free credit reports, and free credit scores.",
                 "Equifax's ability to use acquisitions and joint ventures to expand geographically in emerging markets may be hampered significantly by the breach. Terms will likely be more unfavorable and will take longer for business recentlty acquired like Veda longer to integrate.",
                 "Due to the Equifax breach, one government contract was suspended and other government contracts are under scrutiny.",
                 "The economic uncertainty caused by Brexit could cause Equifax's clients to reduce spending on their solutions on services. The financial impact of Brexit on Equifax depends on agreements the U.K. makes to retain access to EU markets.",
                 "The document claims that hundreds of class action lawsuit were filed after the breach, but a specific number is not provided.",
                 "TransUnion sued Equifax on the grounds of declaratory relief, breach of contract, and anticipatory repudiation of contract.",
                 "The document is states that criminals caused the Equifax security breach, but details on the identity of those criminals is not present in the given document.",
                 "Due to the Equifax breach, various U.S. federal laws and regulations were violated. These include the Fair Credit Reporting Act (FCRA), Dodd-Frank Act, Federal Trade Commission Act (FTC Act), Gramm-Leach-Bliley Act (GLBA), and Credit Repair Organizations Act (CROA).",
                 "Outsourcing vendors have failed to perform at a satisfactory quality despite service level agreements. Because of this, Equifax concludes that outsourcing operations may adversely affect their ability to bring products to market.",
                 "The competitors of Equifax vary in size and offer a diversity of products and service. There are also many competitors in each market that Equifax has products in.",
                 "The percentage of revenue from Equifax's international clients has been increasing since 2015.",
                 "USIS offers three product and service lines. These are Online Information Solutions, Mortgage Solutions, and Financial Marketing Services",
                 "Yes, Equifax ceased advertising for the Global Consumer Solutions line of products and made some of the free. These products provided customers information to understand and monitor their credit and identity.",
                 "Equifax's products and services intend to help customers make more informed decisions with high levels of confidence by leveraging data and insights",
                 "The operating segments of Equifax's business are U.S. Information Solutions (USIS), International, Workforce Solutions, and Global Consumer Solutions.",
                 "Equifax's largest markets are located in the U.S., Asia Pacific, Europe, Canada, and Latin America.",
                 "No, Equifax markets its products and services through their own sales organization.",
                 "Equifax recieves payroll data from over 10,600 companies.",
                 "The Verification Services unit within Equifax provides online verification services for verifiers to verify an employee's employment status and income information.",
                 "The products and services listed as sources of competition for Equifax are consumer credit information solutions and personal solutions products, commercial solutions products, employment and income verification services, and debt collection and recovery management services.",
                 "The GDPR has generally stricter and more comprehensive requirements than other countries that Equifax operates in.",
                 "About 860,000 U.K. consumers were impacted by the breach, while only 19,000 Canadian consumers were impacted.",
                 "Yes, the Indian parliament passed legislation that allows individuals to sue Equifax for damages in the case of a data breach.",
                 "Equifax had approximately 10,300 employees by the end of 2017.",
                 "Equifax does not expect insurance coverage to compensate them for all losses that will occur due to the 2017 cybersecurity breach.",
                 "New product introductions and pricing strategies offered by competitors can decrease their sales and market share. Competitors may develop new products or respond more quickly to new technologies and changing customer requirements.",
                 "There was an increase in companies offering free or low-cost direct to consumer credit services (such as credit scores, reports and monitoring) as part of alternative business models that use such services as a means to introduce consumers to other products and services. To the extent that Equifax's customers choose not to obtain services from us and instead rely on information obtained at no cost or relatively inexpensively from these other sources, our business, financial condition and results of operations may be adversely affected.Due to the 2017 cybersecurity incident and our provision of free services to consumers in connection therewith, we ceased the advertisement and sale of new products in their direct to consumer business, which resulted in a significant decline in revenue in that business. Furthermore, in late January 2018, Equifax began offering a new credit lock service, Lock & Alert, that is free for life and is aimed at empowering consumers to control access to their Equifax credit file directly and quickly from their smartphone or computer. We expect services like our Lock & Alert to continue to increase, thereby eliminating the market for many consumer direct products and services.",
                 "The law would effectively prohibit Equifax from operating as a credit bureau within Ecuador unless they are selected to operate under contract to serve the public authority that the new law tasks with providing such services. These laws and regulations (as well as actions that may be taken by legislatures and regulatory bodies in other countries) and the consequences of any violation could limit their ability to pursue business opportunities they might otherwise consider engaging in, impose additional costs on us, result in significant loss of revenue, result in significant restitution and fines, impact the value of assets they hold,or otherwise adversely affect their business.",
                 "Expect there to be an increased focus on laws and regulations related to our business because of the great public concern in\nthe U.S. with regard to the operation of credit reporting agencies, as well as the collection, use, accuracy, correction and sharing of personal information, which was heightened\nby the 2017 cybersecurity incident.",
                 "IT and security, which require specialized skills, such as maintenance of certain legacy computer systems, data security experts, and analytical modelers.",
                 "1550 Peachtree Street, N.W., Atlanta, Georgia"]

In [49]:
len(page_sources)

40

# RAG System

In [217]:
system_prompt = '''You are tasked with answering queries about Equifax's 10K in 2018 after the 2017 security breach using only the information provided in the most related chunks.
If the answer is not there then return 'no relevant information' in the output and explain what's missing. Otherwise, address the original query
in a concise, accurate, and thorough manner.'''

enhanced_system_prompt = '''You are an elite cybsersecurity analyst with an emphasis on investigating corporate cyber attacks. You have a reputation for being thorough and precise when
providing answers and not making assumptions or jumping to conclusions without sufficient evidence and reasoning.
Your task is to use only the page excerpts from SEC 10-K filings on Equifax to answer general questions about the company and the cyber security breach that occurred.
Return a concise and thorough response if the information provided in the pages can answer the query.

PAGE EXCERPTS:
'''
reminder_prompt = '''REMINDERS:
                     - Ensure that you only answer questions supported by the page excerpts and don't make inferences that are not supported by the text
                     - Recall that 'Brexit' refers to the event of UK splitting from the EU during that time in 2017/2018
                     - If the query is not related to Equifax or the security breach at all or can't be answered with the provided information simply return 'No relevant information' then explain why and don't generate further tokens
                  '''

In [189]:
len(tokenizer.encode(enhanced_system_prompt + reminder_prompt))

212

In [200]:
def get_relevant_chunks(query, k=3, use_qb_index=False):
  test_embedding = get_embedding(query)

  if use_qb_index:
    idx_name = "prophet-equifax-qb"
  else:
    idx_name = "prophet-equifax"

  results = pc.Index(idx_name).query(
      vector=test_embedding,
      top_k=k # Number of results to retrieve
  )

  return results


In [213]:
def get_api_response(system_message, user_message, model="gpt-3.5-turbo", use_reminder_prompt=False):
  messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
  if use_reminder_prompt:
    messages.append({"role": "user", "content" : reminder_prompt})
    # Call GPT model
  response = client.chat.completions.create(
        model=model,
        messages=messages # ,response_format={"type": "json_object"}
    )

    # Extract and return the response content
  return response.choices[0].message.content

In [215]:
def get_top_unique_indices(values, top_n=3):
    # Sort the list based on the second value in descending order
    sorted_values = sorted(values, key=lambda x: x[1], reverse=True)

    # Dictionary to keep track of the unique values and their first index
    unique_indices = {}
    for index, (original_index, value) in enumerate(sorted_values):
        if value not in unique_indices:
            unique_indices[value] = original_index
        if len(unique_indices) == top_n:
            break

    # Get the first indices of the top N unique values
    top_indices = list(unique_indices.values())
    return top_indices

def process_query(system_message, query, pages=pages, use_qb_index=False):
  relevant_data = get_relevant_chunks(query, use_qb_index=False)
  if use_qb_index:
    chunks = [(int(x['id']), x['score']) for x in relevant_data['matches']]
    relevant_qb_data = get_relevant_chunks(query, use_qb_index=True)
    chunks_qb = [(int(x['id'].split('_')[0]), x['score']) for x in relevant_qb_data['matches']]
    chunks += chunks_qb
    chunks = get_top_unique_indices(chunks)
  else:
    chunks = [int(x['id']) for x in relevant_data['matches']]

  chunks = list(set(chunks)) # unique page numbers
  content = str([pages[c] for c in chunks])
  user_message = query + "\n here are the relevant text chunks:\n" + content
  answer = get_api_response(system_message, user_message, use_reminder_prompt=True)

  return chunks, answer

In [177]:
pages[1]

' \nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\n \nWashington, D.C. 20549\n____________________________________\nFORM 10-K\n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended \nDecember 31, 2017\n \nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from                          to \nCommission File Number 001-06605\n____________________________________\nEQUIFAX INC.\n(Exact name of registrant as specified in its charter)\nGeorgia\n \n58-0401110\n(State or other jurisdiction of incorporation or organization)\n \n(I.R.S. Employer Identification No.)\n1550 Peachtree Street, N.W.\n \n \nAtlanta, Georgia\n \n30309\n(Address of principal executive offices)\n \n(Zip Code)\nRegistrant’s telephone number, including area code: 404-885-8000\nSecurities registered pursuant to Section 12(b) of the Act: \nTitle of each class\n \nName of each exchange on whic

# Testing QA

In [218]:
query = "Is Equifax a shell company?"

print(query)
relevant_chunks, response = process_query(system_message=enhanced_system_prompt, query=query, use_qb_index=True)
print(relevant_chunks)
print(response)
print('\n')

Is Equifax a shell company?
[1, 11]
Based on the information provided in the page excerpts from Equifax's SEC 10-K filing, Equifax is not classified as a shell company. The section that discusses the company's status in relation to being a shell company indicates that Equifax is not deemed a shell company.




In [238]:
for num in relevant_chunks:
  print(pages[num])

 
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
 
Washington, D.C. 20549
____________________________________
FORM 10-K
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended 
December 31, 2017
 
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from                          to 
Commission File Number 001-06605
____________________________________
EQUIFAX INC.
(Exact name of registrant as specified in its charter)
Georgia
 
58-0401110
(State or other jurisdiction of incorporation or organization)
 
(I.R.S. Employer Identification No.)
1550 Peachtree Street, N.W.
 
 
Atlanta, Georgia
 
30309
(Address of principal executive offices)
 
(Zip Code)
Registrant’s telephone number, including area code: 404-885-8000
Securities registered pursuant to Section 12(b) of the Act: 
Title of each class
 
Name of each exchange on which registered
Common Stock, $1.25 par val

In [ ]:
#pages[4]

# Question Bank Index

In [42]:
# lot of "missed in middle context" errors, suggesting we should try generate question bank on pages and descriptors/mapping to data ontology. Finally, we can try combining approaches
# compare and contrast each approach in documentation (relative ease of implementation, code maintenance, cost, quantitative metrics, qualitative strengths/weaknesses)


In [43]:
# add Similar Questions Index i.e. generate 5 questions each page is uniquely capable of answering, store it in pineCone, and add to retrieval logic

# Add questions from other parts, especially part II and III
# Get metrics with Vanilla RAG (consider top-1 score, top-3, top-5, eval from GPT(hrock), breakdown eval by question types (one-page specific, multi-page specific, synthesis))
# Error types are wrong source (misgrounded), correct source but wrong conclusion (misinterpretation), incomplete retrieval or poor synthesis for multi-page queries

# Rerun metrics with enhanced index
# ALTERNATIVELY, Focus primarily on specific questions and increasing top-1 score alone (k=1) assuming each question maps to a page (specificty test, subset of benchmark eval)

# Try BM-25 to get pages, then vector embedding search for finer-grained dense search on chunks or another kind of hierarchical retrieval logic + similar questions index on each chunk
# Re-ranker, smaller chunks if time permits
#Query Planning Agent, More sophisticated Router (RAG is just another tool Agent uses)


In [133]:
prompt_q_bank = """Please extract 5-10 questions this page is uniquely capable of answering. If the page is mostly blank or clearly filler, there's
no need to provide anything.

Please return a list of the form
[
    question1,
    question2,
    ....
]

TEXT EXTRACTION:
"""

In [134]:
def parse_questions(input_string):
    # Subset the string between the opening '[' and closing ']'
    start = input_string.find('[') + 1
    end = input_string.rfind(']')
    subset_string = input_string[start:end].strip()

    # Split the subset string by ',\n' to get individual questions
    questions = subset_string.split(',\n')

    # Clean up extra spaces, newlines, and quotation marks
    questions = [q.strip().strip('"').strip() for q in questions]

    return questions

In [248]:
'''page_generated_questions = {} # 5:30 min to generate

for page_num in tqdm(pages):
  u_prompt = prompt_q_bank + pages[page_num]
  response = get_api_response(system_prompt, u_prompt)
  questions = response.strip().strip('[]').strip().split(",\n    ")
  page_generated_questions[page_num] = questions
'''

'page_generated_questions = {} # 5:30 min to generate\n\nfor page_num in tqdm(pages):\n  u_prompt = prompt_q_bank + pages[page_num]\n  response = get_api_response(system_prompt, u_prompt)\n  questions = response.strip().strip(\'[]\').strip().split(",\n    ")\n  page_generated_questions[page_num] = questions\n'

In [ ]:
parse_questions(response)

['Who is the Chief Financial Officer of Equifax Inc. as of March 1, 2018?',
 "What certification was issued in connection with Equifax Inc.'s Annual Report on Form 10-K for the period ended December 31, 2017?",
 'What act mandates the certification mentioned in the text?',
 'What sections of the law are referenced in the certification by John W. Gamble, Jr.?',
 'What criteria does the certification confirm about Equifax Inc.\'s Annual Report for the specified period?",']

In [ ]:
# FINISH THIS (clean out questions, add to index)!
# ADD 5 More HARDER RETRIEVAL on MISSED PAGES questions to Benchmark set (also add a column called "question type") -> (recall/understanding) X (single_page/multi_page)
# Run Vanilla RAG and RAG w/ QB Indices

# Finally, make a note of where the model doesn't perform well and refine prompts ("reminder section as last message", "more thorough system prompt like role of security analyst etc", better sampling of questions)
# Re-run metrics again and store


In [136]:
with open('gpt_generated_question_bank.json', 'r') as file:
    # Step 3: Load the JSON data
    qb = json.load(file)

In [92]:
import re

def split_questions(input_list):
    questions = []

    for item in input_list:
        # Remove leading/trailing whitespace
        item = item.strip()

        # Check if the item has numbered questions
        if re.match(r'^\d+\.', item):
            # Split based on numbered pattern e.g., '1.', '2.', etc.
            split_items = re.split(r'\d+\.\s+', item)
            # Remove any empty strings resulting from the split
            split_items = [q for q in split_items if q.strip()]
            questions.extend(split_items)
        else:
            # Split non-numbered questions by newline followed by space or at least one space
            split_items = re.split(r'\n\s*|\s*\n\s*', item)
            split_items = [q.strip('"').strip() for q in split_items if q.strip()]
            questions.extend(split_items)

    # Final cleaning to ensure no leading/trailing whitespace
    questions = [q.strip() for q in questions if q.strip()]

    return questions


In [26]:
split_questions(qb['9'])

5

In [154]:
# add 'gpt_generated_question_bank.json' embeddings to Pinecone Index
'''
pc.create_index(
      name="prophet-equifax-qb",
      dimension=256,
      metric="cosine",
      spec=ServerlessSpec(
          cloud="aws",
          region="us-west-2"
      )
)

for p_num in qb:
  if int(p_num) < 28:
    if len(qb[p_num]) == 1:
      curr_q_list = split_questions(qb[p_num])
    else:
      curr_q_list = qb[p_num]
    #print(curr_q_list)
    #print(len(curr_q_list))
    for i in range(len(curr_q_list)):
      embedding = get_embedding(curr_q_list[i])
      pc.Index("prophet-equifax-qb").upsert(vectors=[{"id": f"{p_num}_{i}", "values": embedding}])
'''

'\npc.create_index(\n      name="prophet-equifax-qb",\n      dimension=256,\n      metric="cosine",\n      spec=ServerlessSpec(\n          cloud="aws",\n          region="us-west-2"\n      )\n)\n\nfor p_num in qb:\n  if int(p_num) < 28:\n    if len(qb[p_num]) == 1:\n      curr_q_list = split_questions(qb[p_num])\n    else:\n      curr_q_list = qb[p_num]\n    #print(curr_q_list)\n    #print(len(curr_q_list))\n    for i in range(len(curr_q_list)):\n      embedding = get_embedding(curr_q_list[i])\n      pc.Index("prophet-equifax-qb").upsert(vectors=[{"id": f"{p_num}_{i}", "values": embedding}])\n'

21

# Evaluate RAG

In [ ]:
model_responses = []
retrieved_pages = []

for query in tqdm(queries):
   print(query)
   relevant_chunks, response = process_query(system_message=system_prompt, query=query, use_qb_index=True)
   print(relevant_chunks)
   print(response)
   print('\n')

   model_responses.append(response)
   retrieved_pages.append(relevant_chunks)

In [220]:
queries_test = queries
queries_test

['What exactly was the security breach that occurred?',
 'What was the extent of damage done as a result of the incident?',
 'What steps did Equifax take to rectify the hack and regain customer confidence?',
 'How did the attacker succeed and what key information is left out in the document?',
 'Does Equifax operate in Tamil Nadu?',
 'How does Equifax deal with fluctuating currency exchange values to maximize profits?',
 'What are the primary countries in Europe that Equifax operates in?',
 'Which revenue stream is usually the highest in the fourth quarter?',
 "Which government agencies in Australia regulate Equifax's business activity?",
 'What products and services were offered to customers in response to the breach?',
 "How is Equifax's ability to grow its geographical footprint in emerging markets using joint ventures and acquisitions affected by the breach?",
 'How many government contracts were suspended due to the breach?',
 "How would Brexit impact Equifax's bottom-line?",
 'Ho

In [221]:
assert len(queries_test) == len(retrieved_pages) == len(page_sources) == len(model_responses) == len(expert_answers) # "All lists must be of the same length"

In [222]:
# Create a DataFrame
df = pd.DataFrame({
    'questions': queries_test,
    'predicted_sources': retrieved_pages,
    'true_sources': page_sources,
    'model_responses': model_responses,
    'expert_answers': expert_answers
})

# df

In [223]:
def extractJSON(text):
    count = 0
    in_json = False
    start_index = 0

    for index, char in enumerate(text):
        if char == '{':
            if count == 0:
                start_index = index  # Mark the start of a JSON string
                in_json = True
            count += 1
        elif char == '}':
            count -= 1
            if in_json and count == 0:
                # End of the JSON string
                end_index = index + 1  # Include the closing brace
                return text[start_index:end_index]
    return None  # If no JSON found

def extractJSONAsDict(text):
    jsonStr = extractJSON(text)
    if jsonStr:
        return json.loads(jsonStr)
    return {}

In [224]:
GRADING_SYSTEM_PROMPT = """
You are grading the responses between human curated answers and model generated answers
for questions security analysts are asking about recent public company cyberattacks and their repurcussions.

You care about factfulness (70% weight) and not omitting important information (30% weight).
You grade on a 0-100 point scale.
"""

def generate_prompt(q, labeled_response, model_response):
    prompt = f"""

    An analyst asked the following question,

    QUESTION: {q}

    Grade the following model answer based on how closely it
    matches the ground truth answer, especially focusing on the accuracy of details related to security breaches.\n\n
    Ground Truth Answer: {labeled_response}\n\nModel Answer: {model_response}\n\n

    Please return your evaluation as a JSON of the form
    JSON OUTPUT: {{
        "score": str,
        "explanation": str,
    }}
    """
    return prompt

def generate_score(q, labeled_response, model_response):
    prompt = generate_prompt(q, labeled_response, model_response)
    gpt_response = get_api_response(GRADING_SYSTEM_PROMPT, prompt, model='gpt-4o')
    return extractJSONAsDict(gpt_response)

In [225]:
def generate_retrieval_score():
  pass


In [226]:
df.columns

Index(['questions', 'predicted_sources', 'true_sources', 'model_responses',
       'expert_answers'],
      dtype='object')

In [227]:
scores = []
for q, model_resp, true_resp in zip(queries_test, model_responses, expert_answers):
  scores.append(generate_score(q, true_resp, model_resp))

In [228]:
llm_scores = [int(item['score']) for item in scores]
explanations = [item['explanation'] for item in scores]

df['score'] = llm_scores
df['explanation'] = explanations

In [229]:
df.sort_values(['score'])

,questions,predicted_sources,true_sources,model_responses,expert_answers,score,explanation
11,How many government contracts were suspended d...,"[26, 4]",[23],There is no relevant information in the provid...,"Due to the Equifax breach, one government cont...",20,The model answer incorrectly states that there...
26,How many companies does Equifax recieve payrol...,"[3, 6, 15]",[9],The text does not provide the exact number of ...,"Equifax recieves payroll data from over 10,600...",20,The model answer fails to provide the specific...
20,What products and services does USIS offer?,"[9, 7]",[7],"USIS, a business unit of Equifax, offers a var...",USIS offers three product and service lines. T...,30,The model answer does not accurately enumerate...
5,How does Equifax deal with fluctuating currenc...,"[18, 21, 22]",[21],The information provided does not contain spec...,Equifax generally doesn't mitigate risks assoc...,35,The model answer essentially states that there...
2,What steps did Equifax take to rectify the hac...,"[19, 17, 27]","[4, 9, 16]",Equifax took several steps to rectify the hack...,Equifax first acted immediately to stop the in...,45,The model answer did mention some steps taken ...
35,Why does management believe the revenue from d...,"[17, 18, 23]",[19],Management believes the revenue from direct to...,There was an increase in companies offering fr...,50,The model answer captures some relevant aspect...
25,Does Equifax outsource its sales and marketing...,"[6, 23]",[6],"Yes, Equifax outsources various components of ...","No, Equifax markets its products and services ...",50,The model answer provides some correct informa...
3,How did the attacker succeed and what key info...,"[26, 4]","[3, 16]",The document mentions that Equifax acted promp...,The attacker succeeded by exploiting a web app...,55,The model answer contains some relevant inform...
15,Who were the criminals caused the Equifax secu...,"[16, 26, 3]","[3, 4]",The text mentions that Equifax experienced a c...,The document is states that criminals caused t...,60,The model answer provides accurate information...
31,Can Indian customers pursue legal action again...,"[16, 2, 27]",[14],Indian customers might have the legal right to...,"Yes, the Indian parliament passed legislation ...",60,The model answer correctly identifies that cus...


In [237]:
df['score'].mean()

78.25

In [232]:
idx = 3

print(df['model_responses'][idx])
print(df['expert_answers'][idx])
print(df['explanation'][idx])

The document mentions that Equifax acted promptly to notify the affected consumers after the 2017 cybersecurity incident. No evidence was found that the company's core databases were accessed by the attackers. Equifax identified approximately 2.4 million additional U.S. consumers whose information was stolen but were not part of the initially identified group of 145.5 million affected consumers. The company has also cooperated with law enforcement in the criminal investigation into the incident.

However, the exact details of how the attacker succeeded in the breach are not explicitly mentioned in the provided text chunks. Additional information on the specific vulnerabilities or methods used by the attackers to gain unauthorized access to Equifax's systems is missing from the text.
The attacker succeeded by exploiting a web application vulnerability. How that vulnerability was exploited, what the vulnerability was, and why it existed are key pieces of information that are not present 

In [231]:
df.to_csv('RAG_QB_Prompt_ENG_Eval.csv', index=False)

In [ ]:
# vanilla RAG (no prompt engineering or other enhancements): 71.875 mean score
# RAG + Question BANK (QB): 76.925
# RAG + Question BANK + Prompt Engineering: 78.25

#